## Import Libraries Packages, Downloads

In [3]:
import numpy as np
import pandas as pd


## Process Data

In [ ]:
Data = pd.read_csv('NYC_data.csv')
Data.head()

In [ ]:
def process(df): 
    data = df.copy()

    to_binary = {'Yes':1, 'No':0}

    data = data.drop(['Busbreakdown_ID', 'Created_On', 
            'Informed_On','Incident_Number', 
            'Last_Updated_On', 'School_Age_or_PreK', 
            'School_Year', 'Route_Number', 'Schools_Serviced',
            'Run_Type', 'Bus_No', 'Bus_Company_Name'], axis = 1)

    data['Breakdown_or_Running_Late'] = data['Breakdown_or_Running_Late'].map({'Breakdown':1, 'Running Late':0})
    data['Has_Contractor_Notified_Parents'] = data['Has_Contractor_Notified_Parents'].map(to_binary)
    data['Has_Contractor_Notified_Schools'] = data['Has_Contractor_Notified_Schools'].map(to_binary)
    data['Have_You_Alerted_OPT'] = data['Have_You_Alerted_OPT'].map(to_binary)

    data['Occurred_On'] = pd.to_datetime(data['Occurred_On'])
    data['Year'] = data['Occurred_On'].dt.year
    data['Month'] = data['Occurred_On'].dt.month
    data['Day'] = data['Occurred_On'].dt.day
    data['Hour'] = data['Occurred_On'].dt.hour
    data['Mins'] = data['Occurred_On'].dt.minute

    data = data.drop(['Occurred_On'], axis = 1)


    #one hot encode boro
    for boro in data['Boro'].unique():
        data[boro] = (data['Boro'] == boro).astype(int)

    data = data.drop(['Boro'], axis = 1)

    # one hot encode breakdown reason
    for reason in data['Reason'].unique():
        data[reason] = (data['Reason'] == reason).astype(int)

    data = data.drop(['Reason'], axis = 1)
    
    
    data['How_Long_Delayed'] = data.How_Long_Delayed.str.extract('(\d+)')

    return data
